In [0]:
# Change directory to VSCode workspace root so that relative path loads work correctly. Turn this addition off with the DataScience.changeDirOnImportExport setting
# ms-python.python added
import os
try:
	os.chdir(os.path.join(os.getcwd(), '../..'))
	print(os.getcwd())
except:
	pass


 ![](image/2019-11-24-22-12-50.png)

 # 1. 事前に準備済みのK8s環境を確認

 ## 1-1. Cluster

In [33]:
%%bash
az aks show -g rg_bdc -n o9o9aks


Name     Location    ResourceGroup    KubernetesVersion    ProvisioningState    Fqdn
-------  ----------  ---------------  -------------------  -------------------  -----------------------------------------------------
o9o9aks  japaneast   rg_bdc           1.14.6               Succeeded            o9o9aks-rgbdc-745830-7ebfd297.hcp.japaneast.azmk8s.io


In [34]:
%%bash
az aks browse -g rg_bdc --listen-address 0.0.0.0 -n o9o9aks

Process is interrupted.


 ![](image/2019-11-25-15-43-14.png)

 ## 1-2. Node

In [35]:
%%bash
kubectl get node -o wide

NAME                       STATUS   ROLES   AGE   VERSION   INTERNAL-IP   EXTERNAL-IP   OS-IMAGE             KERNEL-VERSION      CONTAINER-RUNTIME
aks-nodepool1-15232104-0   Ready    agent   73d   v1.14.6   10.240.0.4    <none>        Ubuntu 16.04.6 LTS   4.15.0-1063-azure   docker://3.0.6
aks-nodepool1-15232104-1   Ready    agent   73d   v1.14.6   10.240.0.6    <none>        Ubuntu 16.04.6 LTS   4.15.0-1063-azure   docker://3.0.6
aks-nodepool1-15232104-2   Ready    agent   73d   v1.14.6   10.240.0.5    <none>        Ubuntu 16.04.6 LTS   4.15.0-1063-azure   docker://3.0.6


In [36]:
%%bash
az vm list -g MC_RG_BDC_O9O9AKS_JAPANEAST -d

Name                      ResourceGroup                PowerState    PublicIps    Fqdns    Location    Zones
------------------------  ---------------------------  ------------  -----------  -------  ----------  -------
aks-nodepool1-15232104-0  MC_RG_BDC_O9O9AKS_JAPANEAST  VM running                          japaneast
aks-nodepool1-15232104-1  MC_RG_BDC_O9O9AKS_JAPANEAST  VM running                          japaneast
aks-nodepool1-15232104-2  MC_RG_BDC_O9O9AKS_JAPANEAST  VM running                          japaneast


In [37]:
%%bash
kubectl describe node


Name:               aks-nodepool1-15232104-0
Roles:              agent
Labels:             agentpool=nodepool1
                    beta.kubernetes.io/arch=amd64
                    beta.kubernetes.io/instance-type=Standard_B16ms
                    beta.kubernetes.io/os=linux
                    failure-domain.beta.kubernetes.io/region=japaneast
                    failure-domain.beta.kubernetes.io/zone=0
                    kubernetes.azure.com/cluster=MC_rg_bdc_o9o9aks_japaneast
                    kubernetes.azure.com/role=agent
                    kubernetes.io/arch=amd64
                    kubernetes.io/hostname=aks-nodepool1-15232104-0
                    kubernetes.io/os=linux
                    kubernetes.io/role=agent
                    node-role.kubernetes.io/agent=
                    storageprofile=managed
                    storagetier=Premium_LRS
Annotations:        node.alpha.kubernetes.io/ttl: 0
                    volumes.kubernetes.io/controller-managed-attach-det

 # 2. PVC確認

In [38]:
%%bash
cat ~/Jupyter/SQL_Server_on_K8s/FCI/pvc.yaml


kind: StorageClass
apiVersion: storage.k8s.io/v1beta1
metadata:
  name: azure-disk
provisioner: kubernetes.io/azure-disk
parameters:
  storageaccounttype: Standard_LRS
  kind: Managed
---
kind: PersistentVolumeClaim
apiVersion: v1
metadata:
  name: mssql-data
  annotations:
    volume.beta.kubernetes.io/storage-class: azure-disk
spec:
  accessModes:
  - ReadWriteOnce
  resources:
    requests:
      storage: 8Gi

In [39]:
%%bash
kubectl get sc -o wide

NAME                PROVISIONER                AGE
azure-disk          kubernetes.io/azure-disk   17m
default (default)   kubernetes.io/azure-disk   11d
managed-premium     kubernetes.io/azure-disk   11d


In [40]:
%%bash
kubectl describe sc azure-disk


Name:            azure-disk
IsDefaultClass:  No
Annotations:     kubectl.kubernetes.io/last-applied-configuration={"apiVersion":"storage.k8s.io/v1beta1","kind":"StorageClass","metadata":{"annotations":{},"name":"azure-disk"},"parameters":{"kind":"Managed","storageaccounttype":"Standard_LRS"},"provisioner":"kubernetes.io/azure-disk"}

Provisioner:           kubernetes.io/azure-disk
Parameters:            kind=Managed,storageaccounttype=Standard_LRS
AllowVolumeExpansion:  <unset>
MountOptions:          <none>
ReclaimPolicy:         Delete
VolumeBindingMode:     Immediate
Events:                <none>


In [41]:
%%bash
kubectl get pv -o wide -n mssql-fci

NAME                                       CAPACITY   ACCESS MODES   RECLAIM POLICY   STATUS   CLAIM                             STORAGECLASS   REASON   AGE
pvc-96007f59-157a-11ea-8d22-169104339902   8Gi        RWO            Delete           Bound    mssql-fci/mssql-data              azure-disk              17m
pvc-cce7e664-157b-11ea-8d22-169104339902   5Gi        RWO            Delete           Bound    mssql-ag/instance-root-mssql1-0   default                 8m46s
pvc-ce7dd954-157b-11ea-8d22-169104339902   5Gi        RWO            Delete           Bound    mssql-ag/instance-root-mssql2-0   default                 8m38s
pvc-d0480910-157b-11ea-8d22-169104339902   5Gi        RWO            Delete           Bound    mssql-ag/instance-root-mssql3-0   default                 8m35s


In [42]:
%%bash
kubectl describe pv -n mssql-fci


Name:            pvc-96007f59-157a-11ea-8d22-169104339902
Labels:          <none>
Annotations:     pv.kubernetes.io/bound-by-controller: yes
                 pv.kubernetes.io/provisioned-by: kubernetes.io/azure-disk
                 volumehelper.VolumeDynamicallyCreatedByKey: azure-disk-dynamic-provisioner
Finalizers:      [kubernetes.io/pv-protection]
StorageClass:    azure-disk
Status:          Bound
Claim:           mssql-fci/mssql-data
Reclaim Policy:  Delete
Access Modes:    RWO
VolumeMode:      Filesystem
Capacity:        8Gi
Node Affinity:   <none>
Message:         
Source:
    Type:         AzureDisk (an Azure Data Disk mount on the host and bind mount to the pod)
    DiskName:     kubernetes-dynamic-pvc-96007f59-157a-11ea-8d22-169104339902
    DiskURI:      /subscriptions/7458303b-e340-477f-931f-086fd1c1c747/resourceGroups/mc_rg_bdc_o9o9aks_japaneast/providers/Microsoft.Compute/disks/kubernetes-dynamic-pvc-96007f59-157a-11ea-8d22-169104339902
    Kind:         Managed
    FSTy

In [43]:
%%bash
kubectl get pvc -o wide -n mssql-fci

NAME         STATUS   VOLUME                                     CAPACITY   ACCESS MODES   STORAGECLASS   AGE
mssql-data   Bound    pvc-96007f59-157a-11ea-8d22-169104339902   8Gi        RWO            azure-disk     17m


In [44]:
%%bash
kubectl describe pvc -n mssql-fci


Name:          mssql-data
Namespace:     mssql-fci
StorageClass:  azure-disk
Status:        Bound
Volume:        pvc-96007f59-157a-11ea-8d22-169104339902
Labels:        <none>
Annotations:   kubectl.kubernetes.io/last-applied-configuration:
                 {"apiVersion":"v1","kind":"PersistentVolumeClaim","metadata":{"annotations":{"volume.beta.kubernetes.io/storage-class":"azure-disk"},"name...
               pv.kubernetes.io/bind-completed: yes
               pv.kubernetes.io/bound-by-controller: yes
               volume.beta.kubernetes.io/storage-class: azure-disk
               volume.beta.kubernetes.io/storage-provisioner: kubernetes.io/azure-disk
Finalizers:    [kubernetes.io/pvc-protection]
Capacity:      8Gi
Access Modes:  RWO
VolumeMode:    Filesystem
Mounted By:    mssql-deployment-79fdff6b95-7jnt8
Events:
  Type    Reason                 Age   From                         Message
  ----    ------                 ----  ----                         -------
  Normal  Provisio

In [45]:
%%bash
az disk list -g MC_RG_BDC_O9O9AKS_JAPANEAST 


Name                                                                ResourceGroup                Location    Zones    Sku           OsType    SizeGb    ProvisioningState
------------------------------------------------------------------  ---------------------------  ----------  -------  ------------  --------  --------  -------------------
aks-nodepool1-15232104-0_OsDisk_1_233442b330ce48aeb803829051658ad3  MC_RG_BDC_O9O9AKS_JAPANEAST  japaneast            Premium_LRS   Linux     100       Succeeded
aks-nodepool1-15232104-1_OsDisk_1_1f8b8034fcce4ee2b97c024821b6d8bf  MC_RG_BDC_O9O9AKS_JAPANEAST  japaneast            Premium_LRS   Linux     100       Succeeded
aks-nodepool1-15232104-2_OsDisk_1_e0e8ca95bf704caeade77ebf95d47a82  MC_RG_BDC_O9O9AKS_JAPANEAST  japaneast            Premium_LRS   Linux     100       Succeeded
kubernetes-dynamic-pvc-96007f59-157a-11ea-8d22-169104339902         MC_RG_BDC_O9O9AKS_JAPANEAST  japaneast            Standard_LRS            8         Succeeded
kubernetes

 # 3. Pod確認

In [46]:
%%bash
cat ~/Jupyter/SQL_Server_on_K8s/FCI/sql19deployment.yaml


apiVersion: apps/v1beta1
kind: Deployment
metadata:
  name: mssql-deployment
spec:
  replicas: 1
  template:
    metadata:
      labels:
        app: mssql
    spec:
      terminationGracePeriodSeconds: 10
      containers:
      - name: mssql
        image: mcr.microsoft.com/mssql/server:2019-RC1
        ports:
        - containerPort: 1433
        env:
        - name: MSSQL_PID
          value: "Developer"
        - name: ACCEPT_EULA
          value: "Y"
        - name: MSSQL_SA_PASSWORD
          valueFrom:
            secretKeyRef:
              name: mssql
              key: SA_PASSWORD 
        volumeMounts:
        - name: mssqldb
          mountPath: /var/opt/mssql
      volumes:
      - name: mssqldb
        persistentVolumeClaim:
          claimName: mssql-data
---
apiVersion: v1
kind: Service
metadata:
  name: mssql-deployment
spec:
  selector:
    app: mssql
  ports:
    - protocol: TCP
      port: 1433
      targetPort: 1433
  type: LoadBalancer

In [47]:
%%bash
kubectl get pod -o wide -n mssql-fci

NAME                                READY   STATUS    RESTARTS   AGE   IP             NODE                       NOMINATED NODE   READINESS GATES
mssql-deployment-79fdff6b95-7jnt8   1/1     Running   0          16m   10.244.1.130   aks-nodepool1-15232104-2   <none>           <none>


In [48]:
%%bash
kubectl describe pod -n mssql-fci


Name:           mssql-deployment-79fdff6b95-7jnt8
Namespace:      mssql-fci
Priority:       0
Node:           aks-nodepool1-15232104-2/10.240.0.5
Start Time:     Tue, 03 Dec 2019 03:12:52 +0000
Labels:         app=mssql
                pod-template-hash=79fdff6b95
Annotations:    <none>
Status:         Running
IP:             10.244.1.130
IPs:            <none>
Controlled By:  ReplicaSet/mssql-deployment-79fdff6b95
Containers:
  mssql:
    Container ID:   docker://3250d4ffb68e33b25cb36cb5f4e57ec5bd8c1c93bfcd618014bd807f447da71e
    Image:          mcr.microsoft.com/mssql/server:2019-RC1
    Image ID:       docker-pullable://mcr.microsoft.com/mssql/server@sha256:a11facbda1b1cc299d4a37499ef79cd18e38bfb8e5dd7e45cc73670cc07772e5
    Port:           1433/TCP
    Host Port:      0/TCP
    State:          Running
      Started:      Tue, 03 Dec 2019 03:14:04 +0000
    Ready:          True
    Restart Count:  0
    Environment:
      MSSQL_PID:          Developer
      ACCEPT_EULA:        Y
  

In [49]:
%%bash
kubectl get service -o wide -n mssql-fci

NAME               TYPE           CLUSTER-IP     EXTERNAL-IP      PORT(S)          AGE   SELECTOR
mssql-deployment   LoadBalancer   10.0.136.169   52.185.169.168   1433:31492/TCP   16m   app=mssql


In [50]:
%%bash
kubectl describe service -n mssql-fci


Name:                     mssql-deployment
Namespace:                mssql-fci
Labels:                   <none>
Annotations:              kubectl.kubernetes.io/last-applied-configuration:
                            {"apiVersion":"v1","kind":"Service","metadata":{"annotations":{},"name":"mssql-deployment","namespace":"mssql-fci"},"spec":{"ports":[{"por...
Selector:                 app=mssql
Type:                     LoadBalancer
IP:                       10.0.136.169
LoadBalancer Ingress:     52.185.169.168
Port:                     <unset>  1433/TCP
TargetPort:               1433/TCP
NodePort:                 <unset>  31492/TCP
Endpoints:                10.244.1.130:1433
Session Affinity:         None
External Traffic Policy:  Cluster
Events:
  Type    Reason                Age    From                Message
  ----    ------                ----   ----                -------
  Normal  EnsuringLoadBalancer  9m24s  service-controller  Ensuring load balancer
  Normal  EnsuredLoadBalancer 

 # 4. SQL接続テスト

In [51]:
%%bash
kubectl exec mssql-deployment-79fdff6b95-7jnt8 -c mssql -n mssql-fci -- /opt/mssql-tools/bin/sqlcmd -U sa -P $PASSWORD -Q 'SELECT @@VERSION'


------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Microsoft SQL Server 2019 (RC1) - 15.0.1900.25 (X64) 
	Aug 16 2019 14:20:53 
	Copyright (C) 2019 Microsoft Corporation
	Developer Edition (64-bit) on Linux (Ubuntu 16.04.6 LTS) <X64>                                                                                                                      

(1 rows affected)


In [52]:
%%bash
sqlcmd -S 52.185.169.168 -U sa -P $PASSWORD  -Q 'select @@version'


------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Microsoft SQL Server 2019 (RC1) - 15.0.1900.25 (X64) 
	Aug 16 2019 14:20:53 
	Copyright (C) 2019 Microsoft Corporation
	Developer Edition (64-bit) on Linux (Ubuntu 16.04.6 LTS) <X64>                                                                                                                      

(1 rows affected)


In [53]:
%%bash
sqlcmd -S 52.185.169.168 -U sa -P $PASSWORD  -Q 'create database FCIDB'
sqlcmd -S 52.185.169.168 -U sa -P $PASSWORD  -Q 'use FCIDB; create table tab01(id int, name varchar(max)); insert into tab01 values (1, "SQL Server runs on K8s!")'
sqlcmd -S 52.185.169.168 -U sa -P $PASSWORD  -Q 'use FCIDB; select * from tab01'


Changed database context to 'FCIDB'.

(1 rows affected)
Changed database context to 'FCIDB'.
id          name                                                                                                                                                                                                                                                            
----------- ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
          1 SQL Server runs on K8s!                                                                                                                                                                                                                                         

(1 rows affected)


 # 5. Pod障害擬似

In [54]:
%%bash
kubectl get pod -o wide -n mssql-fci


NAME                                READY   STATUS    RESTARTS   AGE   IP             NODE                       NOMINATED NODE   READINESS GATES
mssql-deployment-79fdff6b95-7jnt8   1/1     Running   0          17m   10.244.1.130   aks-nodepool1-15232104-2   <none>           <none>


In [55]:
%%bash
kubectl delete pod mssql-deployment-79fdff6b95-7jnt8 -n mssql-fci


pod "mssql-deployment-79fdff6b95-7jnt8" deleted


In [56]:
%%bash
kubectl get pod -o wide -n mssql-fci


NAME                                READY   STATUS    RESTARTS   AGE   IP             NODE                       NOMINATED NODE   READINESS GATES
mssql-deployment-79fdff6b95-2gkvm   1/1     Running   0          13s   10.244.1.134   aks-nodepool1-15232104-2   <none>           <none>


In [57]:
%%bash
sqlcmd -S 52.185.169.168 -U sa -P $PASSWORD  -Q 'use FCIDB; select * from tab01'

Changed database context to 'FCIDB'.
id          name                                                                                                                                                                                                                                                            
----------- ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
          1 SQL Server runs on K8s!                                                                                                                                                                                                                                         

(1 rows affected)


In [58]:
%%bash
sqlcmd -S 52.185.169.168 -U sa -P $PASSWORD  -Q 'use master; drop database FCIDB'



Changed database context to 'master'.
